In [1]:
from __future__ import division
import os
import pickle
import tensorflow as tf
from helper import create_model
from helper import getData
from helper import imageToVertor
from helper import codeToVertor
from define import codeLength

In [4]:
for index in range(codeLength):

    model_path = 'model/%s/' % index
    model_file_name = os.path.join(model_path, 'model')
    nodes_file_name = os.path.join(model_path, 'nodes.pk')
    if not os.path.exists(nodes_file_name):
        create_model(model_path)

    with tf.Session() as session:
        # 导入模型定义
        saver = tf.train.import_meta_graph(model_file_name + '.meta')
        saver.restore(session, model_file_name)
        nodes = nodes = pickle.load(open(nodes_file_name, "rU"))
        x = session.graph.get_tensor_by_name(nodes['x'])
        y = session.graph.get_tensor_by_name(nodes['y'])
        keep_prob = session.graph.get_tensor_by_name(nodes['keep_prob'])
        loss = session.graph.get_tensor_by_name(nodes['loss'])
        accuracy = session.graph.get_tensor_by_name(nodes['accuracy'])
        optimizer = session.graph.get_operation_by_name(nodes['optimizer'])

        # 训练模型
        for step in range(10):
            imageList, codeList = getData(100)
            codeList = map(lambda x: x[index], codeList)
            x_data = map(imageToVertor, imageList)
            y_data = map(codeToVertor, codeList)
            _, l, a = session.run(
                [optimizer, loss, accuracy], 
                feed_dict={x: x_data, y: y_data, keep_prob: .75})
            saver.save(session, model_file_name)
            print '[%d]: loss: %f accuracy: %f' % (step, l, a)
            if a >= .90:
                break

INFO:tensorflow:Restoring parameters from model/0/model
[0]: loss: 0.337273 accuracy: 0.060000
[1]: loss: 0.330546 accuracy: 0.050000
[2]: loss: 0.329361 accuracy: 0.140000
[3]: loss: 0.342523 accuracy: 0.060000
[4]: loss: 0.342116 accuracy: 0.040000
[5]: loss: 0.333872 accuracy: 0.110000
[6]: loss: 0.328212 accuracy: 0.120000
[7]: loss: 0.330512 accuracy: 0.100000
[8]: loss: 0.332505 accuracy: 0.100000
[9]: loss: 0.338434 accuracy: 0.040000
INFO:tensorflow:Restoring parameters from model/1/model
[0]: loss: 0.338622 accuracy: 0.140000
[1]: loss: 0.322804 accuracy: 0.110000
[2]: loss: 0.334858 accuracy: 0.120000
[3]: loss: 0.341834 accuracy: 0.090000
[4]: loss: 0.340734 accuracy: 0.070000
[5]: loss: 0.336497 accuracy: 0.090000
[6]: loss: 0.328081 accuracy: 0.140000
[7]: loss: 0.330561 accuracy: 0.070000
[8]: loss: 0.331464 accuracy: 0.100000
[9]: loss: 0.331322 accuracy: 0.140000
INFO:tensorflow:Restoring parameters from model/2/model
[0]: loss: 0.336523 accuracy: 0.150000
[1]: loss: 0.